In [1]:
import base64
import numpy as np
import io


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model


from PIL import Image

from flask import request
from flask import jsonify
from flask import Flask

In [2]:
app = Flask(__name__)


In [3]:
labels = ['Cardiomegaly', 
          'Emphysema', 
          'Effusion', 
          'Hernia', 
          'Infiltration', 
          'Mass', 
          'Nodule', 
          'Atelectasis',
          'Pneumothorax',
          'Pleural_Thickening', 
          'Pneumonia', 
          'Fibrosis', 
          'Edema', 
          'Consolidation']

In [4]:
def get_model():
    global model
    # create the base pre-trained model
    base_model = DenseNet121(weights='./models/densenet.hdf5', include_top=False)

    x = base_model.output

    # add a global spatial average pooling layer
    x = GlobalAveragePooling2D()(x)

    # and a logistic layer
    predictions = Dense(len(labels), activation="sigmoid")(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    print(" * Model loaded!")
get_model()

 * Model loaded!


In [5]:

def load_image(img, preprocess=True, H=320, W=320):
    """Load and preprocess image."""
   
    x = image.load_img(img, target_size=(H, W))
    if preprocess:
        x = np.array(x)
        mean = x.mean()
        std = x.std()
        x = x-mean
        x = x/std
        x = np.expand_dims(x, axis=0)
    
    return x    

In [6]:
@app.route("/predict", methods=["POST"])
def predict():
    #getting image from api
    message = request.get_json(force=True)
    #change the image from base64 format to image format
    encoded = message['image']
    decoded = base64.b64decode(encoded)
    image = Image.open(io.BytesIO(decoded))
    
    processed_image = load_image(image)

    prediction = model.predict(processed_image)
    
    preds = {}
    for value in prediction[0]:
        for key in labels:
            preds[key] = value
            labels.remove(key)
            break
    response = preds
    return jsonify(response)